# **ĐỒ ÁN MÔN HỌC: ỨNG DỤNG PHÂN TÍCH DỮ LIỆU THÔNG MINH**

## **Thực hiện:**
|<font size=4> Tên </font>|<font size=4> MSSV </font>|
|-------------------------|----------------------------------|
|<font size=4> Nguyễn Đình Hoàng Nguyên </font>|<font size=4> 19120606 </font>|

# **Giai đoạn 5: Dự đoán**

## **5.0. Tải dữ liệu từ github**

In [1]:
!wget https://raw.githubusercontent.com/HoangNguyen31/Project_Applications-of-Intelligent-Data-Analysis/main/data/preprocessed_apartments.csv

--2024-03-06 07:46:30--  https://raw.githubusercontent.com/HoangNguyen31/Project_Applications-of-Intelligent-Data-Analysis/main/data/preprocessed_apartments.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3368670 (3.2M) [text/plain]
Saving to: ‘preprocessed_apartments.csv’

preprocessed_apartm 100%[===================>]   3.21M  --.-KB/s    in 0.02s   

2024-03-06 07:46:30 (141 MB/s) - ‘preprocessed_apartments.csv’ saved [3368670/3368670]



## **5.1. Import thư viện và Đọc dữ liệu**

In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 7.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats

import time

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv("preprocessed_apartments.csv", encoding="utf-8")
df.head()

,ad_id,list_id,area,area_name,ward,ward_name,street_name,company_ad,apartment_type,size,rooms,toilets,longitude,latitude,deposit,price_million_per_m2,price
0,155094011,114294787,119,Thành phố Thủ Đức,11221.0,Phường An Khánh (Quận 2 cũ),Trần Não,True,"Căn hộ dịch vụ, mini",30.0,1,1.0,106.7304,10.7929,0.0,0.216667,6500000.0
1,154943228,114163716,119,Thành phố Thủ Đức,9341.0,Phường An Phú (Quận 2 cũ),Đường An Phú,True,"Căn hộ dịch vụ, mini",40.0,1,1.0,106.7512,10.8073,0.0,0.250000,10000000.0
2,154792197,114032647,119,Thành phố Thủ Đức,11221.0,Phường An Khánh (Quận 2 cũ),Trần Não,True,"Căn hộ dịch vụ, mini",25.0,1,1.0,106.7304,10.7929,0.0,0.240000,6000000.0
3,152808559,112328722,109,Quận Bình Thạnh,9284.0,Phường 25,D2,False,Chung cư,110.0,3,2.0,106.7160,10.8059,0.0,0.136364,15000000.0
4,154792211,114032660,112,Quận Tân Bình,9301.0,Phường 12,Cộng Hòa,False,Chung cư,74.0,2,2.0,106.6523,10.8030,0.0,0.162162,12000000.0


In [5]:
df.columns

Index(['ad_id', 'list_id', 'area', 'area_name', 'ward', 'ward_name',
       'street_name', 'company_ad', 'apartment_type', 'size', 'rooms',
       'toilets', 'longitude', 'latitude', 'deposit', 'price_million_per_m2',
       'price'],
      dtype='object')

## **5.2. Tiền xử lý dữ liệu**

### **5.2.1. Bỏ thuộc tính dữ liệu không cần thiết**

In [6]:
columns = ["ad_id", "list_id", "area_name", "ward_name", "street_name"]
df.drop(columns, axis=1, inplace=True)

### **5.2.2. Số hóa thuộc tính phân loại**

In [7]:
company_ad_dummies = pd.get_dummies(df["company_ad"], prefix="company")
apartment_type_dummies = pd.get_dummies(df["apartment_type"], prefix="apartment")

df = pd.concat([df, company_ad_dummies, apartment_type_dummies], axis=1)
df.drop(["company_ad", "apartment_type"], axis=1, inplace=True)

### **5.2.3. Chuẩn hóa dữ liệu**

In [8]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)

## **5.3. Dự đoán bằng mô hình học máy**

### **5.3.0. Phân chia dữ liệu**

In [9]:
X = df_scaled.drop("price", axis=1)
y = df_scaled["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
print("Kích thước dữ liệu huấn luyện (X_train):", X_train.shape)
print("Kích thước nhãn huấn luyện (y_train):\t", y_train.shape)

print("Kích thước dữ liệu kiểm tra (X_test):\t", X_test.shape)
print("Kích thước nhãn kiểm tra (y_test):\t", y_test.shape)

Kích thước dữ liệu huấn luyện (X_train): (15863, 17)
Kích thước nhãn huấn luyện (y_train):	 (15863,)
Kích thước dữ liệu kiểm tra (X_test):	 (3966, 17)
Kích thước nhãn kiểm tra (y_test):	 (3966,)


In [11]:
models = []
scores = []
times = []

### **5.3.1. Huấn luyện mô hình**

#### **Linear Regression**

In [12]:
start = time.time()

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

end = time.time()
lr_time = (end-start) / 60

In [13]:
y_pred_lr = lr_model.predict(X_test)
lr_score= np.sqrt(mean_squared_error(y_test, y_pred_lr))

In [14]:
models.append("Linear Regression")
scores.append(lr_score)
times.append(lr_time)

#### **Random Forest**

In [15]:
start = time.time()

rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

end = time.time()
rf_time = (end-start) / 60

In [16]:
y_pred_rf = rf_model.predict(X_test)
rf_score= np.sqrt(mean_squared_error(y_test, y_pred_rf))

In [17]:
models.append("Random Forest")
scores.append(rf_score)
times.append(rf_time)

#### **XGBoost**

In [18]:
start = time.time()

xgb_model = xgb.XGBRegressor(n_estimators=20000, eval_metric="rmse")
xgb_model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_test, y_test)],
              verbose=False,
              early_stopping_rounds=5000)

end = time.time()
xgb_time = (end-start) / 60

In [19]:
y_pred_xgb = xgb_model.predict(X_test)
xgb_score= np.sqrt(mean_squared_error(y_test, y_pred_xgb))

In [20]:
models.append("Xgboost")
scores.append(xgb_score)
times.append(xgb_time)

#### **CatBoost**

In [21]:
start = time.time()

cat_model = CatBoostRegressor(iterations=20000, eval_metric = "RMSE")
cat_model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_test, y_test)],
              use_best_model=True,
              verbose=False)

end = time.time()
cat_time = (end-start) / 60

In [22]:
y_pred_cat = cat_model.predict(X_test)
cat_score= np.sqrt(mean_squared_error(y_test, y_pred_cat))

In [23]:
models.append("CatBoost")
scores.append(cat_score)
times.append(cat_time)

### **5.3.6. Đánh giá kết quả**

In [24]:
models_df = pd.DataFrame()
models_df["models"] = models
models_df["scores"] = scores
models_df["times"] =  times
models_df

,models,scores,times
0,Linear Regression,0.167191,0.000965
1,Random Forest,0.185069,0.178749
2,Xgboost,0.779270,0.536250
3,CatBoost,0.832132,1.488418


Nhận xét:
- Cả 4 thuật toán đều mô hình hóa thành công dữ liệu.
- Mô hình `Xgboost` và `CatBoost` cho ra kết quả đưa ra kết quả khá tốt, cũng như thời gian chạy lâu hơn.
- Mô hình đưa ra kết quả tốt nhất là `Random Forest` và `Linear Regression` cùng thời gian huấn luyện là nhanh nhất. Đây là 2 mô hình phù hợp với bài toán và dữ liệu đặt ra.